### Project Overview

I extracted the food items that I purchased at the REWE supermarket in Frankfurt, Germany from **11/05/2023 to 02/06/2023** using PDF receipts downloaded from my personal REWE account.

The goal is to parse these receipts and extract structured data, such as item names, prices, and shopping dates for personal expenditure tracking.

---

### Step 1: Import the Required Packages

We begin by importing the necessary Python libraries:

- `pdfplumber` to extract text from the PDF receipts  
- `glob` to locate all PDF files in a folder using wildcards (`*`, `?`)  
- Standard libraries like `os`, `re`, and `datetime`  
- `pandas` for data manipulation and analysis

In [2]:
import warnings
warnings.filterwarnings("ignore")
import sys
import os
import re
import glob
import pdfplumber
import pandas as pd
from datetime import datetime
from contextlib import contextmanager
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)

### Step 3: Define the File Path

We now specify the folder containing all downloaded REWE supermarket receipt PDFs.  
For security reasons, the actual path is masked in this notebook.

In [4]:
pdf_folder = "C:/Users/rugge/Dropbox/Personal Portfolio/Fitness/Food Expenditure"  

### Step 5: Use Regex Patterns to Extract Item Lines and Dates

We define two regular expressions to extract structured information from the receipts:

1. `item_price_pattern`  
   This pattern matches lines in the receipt that contain a product name (in uppercase German text) followed by a price (e.g., 1,49).  
   It allows for special characters like umlauts (Ä, Ö, Ü, ß), punctuation, and European decimal formatting with a comma.

2. `date_pattern`  
   This pattern extracts the shopping date from the filename.  
   The format we expect is "vom DD.MM.YYYY" — which is how REWE names their downloaded PDF receipts.


In [6]:
item_price_pattern = re.compile(
    r"^([A-ZÄÖÜß0-9 .\-/%]+?)\s+([0-9]{1,2},[0-9]{2})\s*[ABВ]?$"
)

date_pattern = re.compile(r"vom\s(\d{2}\.\d{2}\.\d{4})")

### Extracting Supermarket Receipt Data from PDF Files

We load the PDF files from a folder containing supermarket receipts and extract key information from each one.

We begin by creating an empty list called `all_items`, and for each valid line in a receipt, we append a dictionary with the following details:

-  **File name** — the name of the PDF file
-  **Date** — the extracted shopping date from the filename (if available)
-  **Item** — the product name, usually in German
-  **Total paid** — the amount paid for the item

See the code cell below for detailed comments on each step in the process.

In [8]:
all_items = []

for pdf_path in glob.glob(os.path.join(pdf_folder, "*.pdf")):
    file_name = os.path.basename(pdf_path)

    date_match = date_pattern.search(file_name)
    if date_match:
        shopping_date_str = date_match.group(1)
        shopping_date = datetime.strptime(shopping_date_str, "%d.%m.%Y")
    else:
        shopping_date = None

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page.cropbox = page.mediabox
                text = page.extract_text()
                if not text:
                    continue

                for line in text.split('\n'):
                    if any(skip_word in line for skip_word in [
                        "SUMME", "Ges.", "EUR", "Bezahlung", "Betrag", "Steuer", "Gesamtbetrag"
                    ]):
                        continue

                    match = item_price_pattern.match(line.strip())
                    if match:
                        item = match.group(1).strip()
                        price = float(match.group(2).replace(",", "."))
                        all_items.append({
                            "file": file_name,
                            "date": shopping_date,
                            "item": item,
                            "total_paid": price
                        })
    except Exception as e:
        print(f"Error processing {file_name}: {e}")

### Step 6: Convert Extracted Items to a DataFrame

Now that we've collected all the extracted item details into the `all_items` list, we can convert it into a structured Pandas DataFrame for analysis.


In [10]:
df = pd.DataFrame(all_items)

### Step 7: View the DataFrame

Let's display the DataFrame to inspect the first few rows.


In [12]:
df

,file,date,item,total_paid
0,Dein REWE eBon vom 01.06.2023.pdf,2023-06-01,HA-BRUSTFILET,6.30
1,Dein REWE eBon vom 01.06.2023.pdf,2023-06-01,BIO MEHRKORNBR.,2.79
2,Dein REWE eBon vom 01.06.2023.pdf,2023-06-01,RUSTIKAL KNAECKE,2.39
3,Dein REWE eBon vom 01.06.2023.pdf,2023-06-01,BABY SPINAT,3.98
4,Dein REWE eBon vom 01.06.2023.pdf,2023-06-01,TOMATE,6.98
...,...,...,...,...
64,Dein REWE eBon vom 27.05.2023.pdf,2023-05-27,DUSCHE ICE KICK,1.19
65,Dein REWE eBon vom 30.05.2023 (1).pdf,2023-05-30,SPINAT 300G,2.50
66,Dein REWE eBon vom 30.05.2023.pdf,2023-05-30,HA-BRUSTFILET,5.37
67,Dein REWE eBon vom 30.05.2023.pdf,2023-05-30,BIO FRISCHKAESE,1.29


### Step 8: Filter Out Non-Food Items

We only want to analyze food-related purchases from the supermarket receipts.  
The list below contains non-food items (in German) that were manually identified — such as cleaning supplies, personal hygiene products, and kitchen accessories. We create a new dataframe called df_food


In [14]:
nonfood_items = ['5 VERSCHL.-KLIPS', 
                 'SENS.WHITE ZC', 
                 'VOLLWASCHMITTEL', 
                 'TO-GO SALATDOSE', 
                 'SCHEUERSPIRALE', 
                 'ALLZWECKTUECHER', 
                 'OXI ACTIONPULVER', 
                 'FRO.LIMO.SPUEL.', 
                 'JA TOIPA 3LG', 
                 'DUSCHE ICE KICK']

df_food = df[~df['item'].isin(nonfood_items)]

### Step 9: Verify That Only Food Items Remain

To ensure our filtering worked correctly, we compare the number of rows in the original DataFrame (`df`) and the filtered DataFrame (`df_food`).

If the filtering was successful, `df_food` should contain fewer rows than `df`.

In [16]:
print('The number of rows in df is',len(df))
print('The number of rows in df_food is',len(df_food))


The number of rows in df is 69
The number of rows in df_food is 59


### Step 10: Manually Label Food Categories

To categorize the food items into broader food groups, we define a mapping dictionary called `label_map`.

Each key represents an item name (in uppercase German), and the value represents the food group it belongs to.


In [18]:
label_map = {
    "HA-BRUSTFILET": "Poultry",
    "BABY SPINAT": "Fruits and Vegetables",
    "GEMUESE FOND": "Fruits and Vegetables",
    "TOMATE": "Fruits and Vegetables",
    "SPINAT 300G": "Fruits and Vegetables",
    "ZITRONE": "Fruits and Vegetables",
    "BIO FRISCHKAESE": "Bread, Dairy, Nuts and Rice",
    "BIO MEHRKORNBR.": "Bread, Dairy, Nuts and Rice",
    "RUSTIKAL KNAECKE": "Bread, Dairy, Nuts and Rice",
    "KOKOSMILCH LIGHT": "Bread, Dairy, Nuts and Rice",
    "NUSSKERNMISCHUNG": "Bread, Dairy, Nuts and Rice"
}



### Step 11: Categorize Items and Analyze Spending Patterns

We now assign each food item a category label using the `label_map` dictionary defined earlier.  
This new column, `food_label`, allows us to group spending by category.

If an item does not appear in the mapping, we assign it the default label `"Other"`.

In [20]:
df_food['food_label'] = df_food['item'].map(label_map).fillna("Other")

### Step 12: Calculate Total Spend by Category

We group the food DataFrame by the `food_label` column and sum the `total_paid` values to compute how much was spent in each category.

This gives us a breakdown of spending across categories like Poultry, Fruits and Vegetables, and Bready/Dairy items.


In [22]:
df_groupspend = df_food.groupby('food_label')['total_paid'].sum().reset_index()

df_groupspend

,food_label,total_paid
0,"Bread, Dairy, Nuts and Rice",50.6
1,Fruits and Vegetables,79.7
2,Poultry,115.5


### Step 13: Calculate Category Spending Shares

To understand how much each food category contributed to overall spending, we calculate the **spending share** for each group.

Steps:
- Compute the total food spend across all items.
- For each category, divide its total by the overall spend and multiply by 100 to get the percentage.
- Round the results to make the output cleaner.


In [24]:
total_spend = df_food['total_paid'].sum()

df_groupspend['share'] = 100*df_groupspend['total_paid']/ total_spend

df_groupspend[['food_label','total_paid','share']].round(0)

,food_label,total_paid,share
0,"Bread, Dairy, Nuts and Rice",51.0,21.0
1,Fruits and Vegetables,80.0,32.0
2,Poultry,116.0,47.0


### Step 14: Estimate Weekly Food Spend

To estimate how much was spent on food per week, we calculate a **pro-rata weekly average** based on the number of days covered by the receipts.

In this case, the data spans 22 days. We scale the total spend to a 7-day week:

In [26]:
weekly_spend = (total_spend / 22 * 7)

print(f"\nWeekly spend: €{weekly_spend:.0f}")


Weekly spend: €78
